# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [28]:
!cat publications.tsv








2017/06/01	Sistem Klasifikasi Variabel Tingkat Penerimaan Konsumen Terhadap Mobil Menggunakan Metode Random Forest	Jurnal Teknik Elektro	Pengembangan sistem pengklasifikasi tingkat penerimaan mobil oleh konsumen menggunakan metode Random Forest (RF).	"Nugroho, Y.S., Emiliyawati, N. (2017). ""Sistem Klasifikasi Variabel Tingkat Penerimaan Konsumen Terhadap Mobil Menggunakan Metode Random Forest"". Jurnal Teknik Elektro. 9(1)."	aplikasi-penerimaan-mobil-RF	https://journal.unnes.ac.id/nju/index.php/jte/article/view/10452/6660

## Import pandas

We are using the very handy pandas library for dataframes.

In [29]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [30]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, encoding='latin-1')
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2011/07/01,Perkembangan Pengetahuan Anak Usia Dini Melalu...,Jurnal KomuniTi,Pelaksanaan penelitian ini bertujuan untuk men...,"Nugroho, Y.S. (2011). ""Perkembangan Pengetahua...",perkembangan-anak-usia-dini,http://journals.ums.ac.id/index.php/komuniti/a...
1,2013/09/02,"Analisa Kecepatan Proxy Squid, Safesquid dan P...",Jurnal Emitor,Penelitian ini untuk membandingkan beberapa si...,"Al Irsyadi, F.Y., Nugroho, Y.S., Wuryanto, D. ...",analisis-squid-safesquid,https://publikasiilmiah.ums.ac.id/xmlui/handle...
2,2013/09/01,Seleksi Sekolah Menengah Lanjutan Menggunakan ...,Jurnal KomuniTi,Penelitian ini dilakukan dalam rangka mengemba...,"Nugroho, Y.S., Ulinuha, A., Aji, N.N.S. (2013)...",seleksi-sml-dengan-ahp,https://publikasiilmiah.ums.ac.id/handle/11617...
3,2014/03/01,Klasifikasi Masa Studi Mahasiswa Fakultas Komu...,Jurnal KomuniTi,Penelitian ini dilakukan untuk memanfaatkan da...,"Nugroho, Y.S., Setyawan. (2014). ""Klasifikasi ...",klasifikasi-mhs-fki-c45,http://journals.ums.ac.id/index.php/komuniti/a...
4,2015/12/01,Aplikasi Pemrediksi Masa Studi dan Predikat Ke...,Jurnal Khazanah Informatika,Penelitian ini dilakukan untuk membuat suatu a...,"Nurrohmat, M.A., Nugroho, Y.S. (2015). ""Aplika...",aplikasi-pemrediksi-kelulusan-mhs,http://journals.ums.ac.id/index.php/khif/artic...
5,2015/12/01,Klasifikasi dan Klastering Penjurusan Siswa SM...,Jurnal Khazanah Informatika,"Penerapan algoritma C4.5, Naive Bayes dan K-Me...","Nugroho, Y.S., Haryati, S.N. (2015). ""Klasifik...",klasifikasi-penentuan-jurusan-sekolah,http://journals.ums.ac.id/index.php/khif/artic...
6,2017/06/01,Sistem Klasifikasi Variabel Tingkat Penerimaan...,Jurnal Teknik Elektro,Pengembangan sistem pengklasifikasi tingkat pe...,"Nugroho, Y.S., Emiliyawati, N. (2017). ""Sistem...",aplikasi-penerimaan-mobil-RF,https://journal.unnes.ac.id/nju/index.php/jte/...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [31]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [32]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [33]:
!ls ../_publications/

01-aplikasi-pemrediksi-kelulusan-mhs.md
01-aplikasi-penerimaan-mobil-RF.md
01-klasifikasi-mhs-fki-c45.md
01-klasifikasi-penentuan-jurusan-sekolah.md
01-perkembangan-anak-usia-dini.md
01-seleksi-sml-dengan-ahp.md
02-analisis-squid-safesquid.md


In [34]:
!cat ../_publications/01-aplikasi-pemrediksi-kelulusan-mhs.md

---
title: "Aplikasi Pemrediksi Masa Studi dan Predikat Kelulusan Mahasiswa Informatika Universitas Muhammadiyah Surakarta Menggunakan Metode Naive Bayes."
collection: publications
permalink: /publication/2015/12/01-aplikasi-pemrediksi-kelulusan-mhs
excerpt: 'Penelitian ini dilakukan untuk membuat suatu aplikasi untuk melakukan prediksi terhadap lama studi dan predikat kelulusan mahasiswa dengan menerapkan metode Naive Bayes.'
date: 2015/12/01
venue: 'Jurnal Khazanah Informatika'
paperurl: 'http://journals.ums.ac.id/index.php/khif/article/view/1179/1028'
citation: 'Nurrohmat, M.A., Nugroho, Y.S. (2015). &quot;Aplikasi Pemrediksi Masa Studi dan Predikat Kelulusan Mahasiswa Informatika Universitas Muhammadiyah Surakarta Menggunakan Metode Naive Bayes&quot;. Khazanah Informatika. 1(1).'
---
Penelitian ini dilakukan untuk membuat suatu aplikasi untuk melakukan prediksi terhadap lama studi dan predikat kelulusan mahasiswa dengan menerapkan metode Naive Bayes.

[Download paper here](http://j